## <center> TP analyse des opinions dans les Tweets </center>##
#### <center> Elaboré par : Mohamed DHAOUI  - MS Big Data  </center>####

L’objectif de ce TP est d’analyser un corpus de tweets en fonction des opinions exprimées (positif/-
négatif). Le langage à utiliser est Python.

La base des tweets à analyser contient 498 tweets annotés manuellement. La base propose 6 champs correspondant aux informations suivantes :
1. la polarité du tweet : Chaque tweet est accompagné d’un score pouvant être égal à 0 (négatif), 2
(neutre) ou 4 (positif).
2. l’identifiant du tweet (2087)
3. la date du tweet (Sat May 16 23 :58 :44 UTC 2009)
4. la requête associée (lyx). Si pas de requête la valeur est NO_ QUERY.
5. l’utilisateur qui a tweeté (robotickilldozr)
6. le texte du tweet(Lyx is cool)


### Importation des packages 

In [552]:
# coding: utf8
import pandas as pd
import numpy as np
import re
import nltk as nltk
import re
from nltk.corpus import wordnet as wn
import csv



from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

import pandas as pd
from sklearn.decomposition.pca import PCA
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from gensim.models import Word2Vec
import html.entities


## 1- Prétraitements

Dans cette partie , on va essayer de nettoyer les tweets et supprimer les caractères spéciaux susceptibles de nuire à la mise en place des méthodes d’analyse d’opinions. Pour ce faire , on va commencer par récuperer le fichier de tweet , ajouter les noms de colonnes et ensuite faire les étapes suivantes : 
- récupérer le texte associé
- segmenter en tokens
- supprimer les urls
- nettoyer les caractères inhérents à la structure d’un tweet
- corriger les abréviations et les spécificités langagières des tweets à l’aide du dictionnaire DicoSlang (fichier SlangLookupTable.txt d

On commence par lire le fichier de tweets : 

In [451]:
df = pd.read_csv('testdata.csv', sep = ',', header=None,encoding='latin1')


In [452]:
df.columns=["score","id","date","req","user","content"]

In [453]:
df.head()

,score,id,date,req,user,content
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...


On crée ensuite des copies de dataset pour une utilisation future 

In [454]:
df_init=df.copy()


In [455]:
df_temp=df.copy()

df_temp['content'] = df_temp['content'].apply(lambda x: re.sub('https?://[A-Za-z0-9./]+','',x))
#df_temp['content'] = df_temp['content'].apply(lambda x: re.sub('([#])|([^a-zA-Z])',' ',x))


In [461]:
df_temp.content[1]

'Reading my kindle2...  Love it... Lee childs is good read.'

#### Prétraitement : ‘@’mention

Maintenant , on va essayer de traiter les mentions @ . Bien que ces mentions contiennent des informations ( la personne qui  a tweeté ou qui concerné par le tweet ou ...) , cela n'est pas pertinent dans le modèle d'analyse de sentiment . De ce fait ,on va les supprimer en utilisant le regex 


Avant de supprimer les "@" , essayant de récuperer le nombre de mentions "@..." dans tout le corpus 

In [465]:
nb_att=sum(df['content'].apply(lambda x: len(re.findall(r'@[A-Za-z0-9]+', x))))
print( "Nombre de mention '@' est %s" %nb_att)



Nombre de mention '@' est 124


C'est un peu étrange car on s'attendait à que le nombre de mentions de @ soit comparable au nombre de tweet ... Supprimons maintenant ces mentions

In [458]:
df.content[0]

'@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.'

In [466]:

df['content'] = df['content'].apply(lambda x: re.sub(r'@[A-Za-z0-9]+','',x))
#[^a-zA-Z]", " "


In [467]:
df.content[0]

' I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.'

#### Prétraitement: URL links
Ensuite , on va supprimer les url links , ces url contiennent bien evidemment des informations mais qui ne sont pas utiles à l'analyse de sentiment 

In [468]:
df['content'] = df['content'].apply(lambda x: re.sub('https?://[A-Za-z0-9./]+','',x))


In [470]:
df.content[17]

'i love lebron. '

#### Prétraitement : Les hashtags


Commençons par calculer le nombre de hashtag dans le corpus

In [472]:
nb_hash=sum(df['content'].apply(lambda x: len(re.findall(r'([#])|([^a-zA-Z])', x))))
print( "Nombre de hashtag  est %s" %nb_hash)

Nombre de hashtag  est 8477


On s'attendait à que le nombre de hashtag soit aussi élévé car il fait partie des entités les plus trouvées dans les tweets . Malgré qu'ils peuvent apporter une information utile au sentiment , on préfère en premier lieu supprimer les hashtags pour facilier le traitement.Supprimons maintenant les Hashtags

In [473]:


df['content'] = df['content'].apply(lambda x: re.sub('([#])|([^a-zA-Z])',' ',x))



#### Prétraitement : Double espace

Ensuite , on va supprimer les double space des tweets

In [295]:
df['content'] = df['content'].apply(lambda x: re.sub(' +', ' ',x))
df.content[2]

'Ok first assesment of the kindle it fucking rocks '

#### Calcul du nombre de caractère inhérents à la structure de tweet

Ici , on va supposer que le nombre d’occurrences des caractères inhérents à la structure de tweet est le nombre de hashtag plus nombre de mentions @ . De ce fait ce nombre est 8477+124 = **8601** caractères

#### Calcul du nombre hashtag


Le nombre de hashtag a été déjà calculer et il est égal à **8477**

#### Prétraitement : correction des abréviations et des spécificités langagières des tweets 

Comme on a mentionné au début de cette partie , pour améliorer le nettoyage du fichier , on va utiliser un dictionnaire contenant les corrections des abréviations des tweets 

Lecture des fichiers des abréviations

In [477]:
dictfile="SlangLookupTable.txt"
with open(dictfile) as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)
    


Transformation du fichier en dataframe

In [478]:
df_dict = pd.DataFrame(d)
df_dict.columns=['abv','mean']


In [479]:
df_dict.head(5)

,abv,mean
0,121,one to one
1,a/s/l,"age, sex, location"
2,adn,any day now
3,afaik,as far as I know
4,afk,away from keyboard


Maintenant , on va créer une fonction `correcttabv` qui permet de reperer les abréviations dans les tweets ( soit à la fin , soit , au début , soit au milieu ) et les corriger 

In [480]:
testtext="adn i do not know afk 121"

In [481]:
def correctabv(text) :
    for i in df_dict.abv :
        text=text.replace(" "+ i+ " ", " ")
        text=text.replace(" "+ i+ ",", " ")
        text=text.replace(" "+ i+ ".", " ")
        text=text.replace(" "+ i+ "!", " ")
        
        if testtext.find(i)+len(i) ==len(text):
            text=text.replace(i, " ")
        if testtext.find(i)==0:
            text=text.replace(i, " ")

    return text

In [483]:
correctabv(testtext)

'  i do not know  '

Appliquons maintenant cette fonction au dataset 

In [484]:
df['content'] =df['content'].apply(lambda x: correctabv(x))


#### Prétraitment :  tokenize 
La tokenization est la tache de splitter les une chaines de caractères en plusieurs mots . Cela peut se faire via la fonction split classique , mais afin de tenir compte de nature grammaticale des mots et le contexte dans lequel il a été employé , il vaut mieux utiliser la fonction `word_tokenize` de nltk  

In [491]:
preProcessedTweet = df['content'].apply(lambda x: nltk.word_tokenize(x))


In [492]:
preProcessedTweet[0:5]

0    [I, loooooooovvvvvveee, my, Kindle, Not, that,...
1    [Reading, my, kindle, Love, it, Lee, childs, i...
2    [Ok, first, assesment, of, the, kindle, it, fu...
3    [You, ll, love, your, Kindle, I, ve, had, mine...
4    [Fair, enough, But, i, have, the, Kindle, and,...
Name: content, dtype: object

### 2- Etiquetage grammatical


Maintenant on va s'interesser à la nature grammaticale des mots . Cela va nous servira après pour filtrer les mots .
Pour ce faire, on va developper une fonction capable de déterminer la catégorie grammaticale (POS : Part Of Speech) de chaque mot du tweet 

In [83]:
taggedTweet=preProcessedTweet.apply(lambda x: nltk.pos_tag(x))
taggedTweet[0:5]

0    [(I, PRP), (loooooooovvvvvveee, VBP), (my, PRP...
1    [(Reading, VBG), (my, PRP$), (kindle, NN), (Lo...
2    [(Ok, NNP), (first, JJ), (assesment, NN), (of,...
3    [(You, PRP), (ll, VBP), (love, VB), (your, PRP...
4    [(Fair, NNP), (enough, RB), (But, CC), (i, NNS...
Name: content, dtype: object

On développe ensuite une fonction permettant de compter le nombre de verbes dans un tweet : 

In [86]:
listverb_ind=["VB","VBD","VBG","VBN","VBP","VBZ"]

def countverb(listtoken):
    return np.sum([1 for i,j in listtoken if j in listverb_ind])

On affiche ensuite le nombre de verbes dans le corpus 

In [494]:
nb_verbs=sum(taggedTweet.apply(lambda x: countverb(x)))
print( "Nombre de verbes est %s" %nb_verbs)

Nombre de verbes est 1119.0


### 3 . Algorithme de détection v1 : appel au dictionnaire Sentiwordnet


Dans cette partie , on a exploiter quelques fonctionalités intéressantes de NLTK : On utilisera la base de données wordnet qui permet d'accéder à l’ensemble des synsets qui sont liés à un mot donné à l’aide d’une commande simple sous Python. 

Dans cette étape , on fera les taches suivantes : 
- Récupération uniquement des mots correspondant à des adjectifs, noms, adverbes et verbes
- Calculer pour chaque mot du tweet les scores associés à leur premier synset
- Calculer pour chaque tweet la somme des scores positifs et négatifs des SentiSynsets du tweet,
— Comparer la somme des scores positifs et des scores négatifs de chaque tweet pour décider de la
classe à associer au tweet.


Pour commencer , on définit une liste contenant les tag à garder dans le tweet

In [312]:

listkeep=["JJ","FW","JJR","JJS","NN","NNS","NNP","NNPS","RB","RBR","RBS","VB","VBD","VBG","VBN","VBP","VBZ"]


Ensuite , on définit une fonction `extractscoremot` qui renvoie les scores d'un mot donnée . Après on définit la fonction `scoretweet` qui prend comme input un tagwords de tweet , filtre les elements correspondant à {adj,noms,adv,verbes} , calcule le score de chaque mot et enfin renvoyer la somme des scores positifs et négatifs

In [496]:


def extractscoremot (mot):
    tt=wn.synsets(mot)
    if len(wn.synsets(mot)) > 0 :
        tt=tt[0].name() # à exploiter les autre options
        s=swn.senti_synset(tt)
        return s.pos_score(),s.neg_score()
    else :
        return 0,0

def scoretweet (tweettag):
    
    listnouns=[ x for (x,y) in tweettag if y in listkeep]
    return {'pos':np.sum([ extractscoremot(mot)[0] for mot in listnouns]),'neg':np.sum([ extractscoremot(mot)[1] for mot in listnouns])}

Un exemple d'output correspondant à un tweettags : 

In [497]:
scoretweet(taggedTweet[0])

{'pos': 1.125, 'neg': 1.25}

On applique ensuite cette fonction à notre corpus prétraité

In [316]:
taggedTweetscore=taggedTweet.apply(lambda x: scoretweet(x))


On définit une fonction qui permettent de classer les tweet en {0,2,4}

In [498]:
def classscore(dictscore):
    if dictscore['pos'] > dictscore['neg'] :
        return [dictscore['pos'],dictscore['neg'],4]
    elif dictscore['pos'] < dictscore['neg'] : return [dictscore['pos'],dictscore['neg'],0]
    else : return  [dictscore['pos'],dictscore['neg'],2]


In [499]:
taggedTweetClass=taggedTweetscore.apply( lambda x: classscore(x))

L'output de cette fonction est le suivant  :

In [501]:
taggedTweetClass[0:5]

0     [8.125, 1.25, 4]
1    [6.625, 0.125, 4]
2        [4.0, 7.0, 0]
3       [14.0, 3.0, 4]
4        [7.0, 2.0, 4]
Name: content, dtype: object

On récupérer ensuite la classe prédite pour chaque tweet et on calcule enfin la matrice de confusion :

In [502]:
predctedscore=taggedTweetClass.apply(lambda x:x[2])

In [503]:
df_confusion = pd.crosstab(df.score, predctedscore, rownames=['Actual'], colnames=['Predicted'], margins=True)


In [504]:
df_confusion

Predicted,0,2,4,All
Actual,,,,
0,65,7,105,177
2,15,12,112,139
4,10,5,167,182
All,90,24,384,498


Ici on voit bien que le nombre de tweet positif correctement prédits est **167**  , ce qui est très moyen comme taux car on prédit beaucoup de positifs par rapports au réel ( 105 tweets neutres & 112 tweets négatifs sont classé positifs via notre algorithme) . On constate aussi que le nombre de tweet de classe 0  et dont la prédiction donne 4 est 105 , de ce fait , il y a un problème de distinction entre la classe neutre et la classe positive .On voit également qu'on a des erreurs de prédictions élévés sur les autres classes. Essayons de claculer la précision globale de l'algorithme .

In [508]:
score=(df_confusion[0:1][0].values[0] +df_confusion[1:2].values[0][1] +df_confusion[2:3].values[0][2])/df_confusion[3:4].values[0][3]
print( "Le score de la première méthode est %.2f" %score)

Le score de la première méthode est 0.49


Le score est très faible et il reste encore du travail à effectuer ...

### 4- Algorithme de détection v2 : gestion de la négation et des modifieurs

Dans cette partie , on va essayer de tenir compte des négations et des modifieurs dans le scoring des tweets
On aura  besoin de : 
- La liste des mots en anglais correspondant à des négations 
- La liste des mots correspondant aux modifieurs 

On implémentera une fonctionqui ,pour chaque mot, effectue les opérations suivantes :
- multiplie par 2 le score négatif et le score positif associés au mot si le mot précédent est un modifieur ;
- utilise uniquement le score négatif du mot pour le score positif global du tweet et le score positif
du mot pour le score négatif global du tweet si le mot précédent est une négation.


On commence par lire le dictionnaire de modifiers et des négations et les transformer en dataframe

In [513]:
dictfile="BoosterWordList.txt"

with open(dictfile) as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)
modifiers=pd.DataFrame(d)
modifiers.columns=["modifier","att"]
modifierslist=modifiers.modifier.tolist()   
modifiers.head()


,modifier,att
0,absolutely,1
1,definitely,1
2,extremely,2
3,fuckin,2
4,fucking,2


Pour les négations , on a pris les mots du dictionnaire fourni avec le TP et on les a mis dans une liste

In [105]:
negatives=["aren't","arent","can't","cannot","cant","don't","dont","isn't","isnt","never","not","won't","wont","wouldn't","wouldnt"]


On développe ensuite une fonction qui prend un mot et son prédecesseur et renvoie les scores positif et négatif de tweet en tenant compte des régles expliquées au début de la partie 

In [329]:

def extractscoremotmodif (mot,prec):
    tt=wn.synsets(mot)
    coef=1
    if len(wn.synsets(mot)) > 0 :
        if prec in (modifierslist): 
            coef=2
        tt=tt[0].name() # à exploiter les autre options
        s=swn.senti_synset(tt)
        if prec in negatives :
            return coef*s.neg_score(),coef*s.pos_score(),1
            
        else :
            return coef*s.pos_score(),coef*s.neg_score(),0
    
    else :
        return 0,0,0


On implémente ensuite une fonction `scoretweetmodif` qui permet de scorer un tweettags en appelant la fonction précedement définie 

In [111]:

def scoretweetmodif (tweettag):

    scorepos=[]
    listnouns=[ x for (x,y) in tweettag if y in listkeep]

    for ind,mot in enumerate(listnouns) :
        if ind > 0 :
            score = extractscoremotmodif (mot,listnouns[ind-1])
        else :
            score =extractscoremot(mot)
            score=(score[0],score[1],0)
        scorepos.append(score)
    return {'pos':np.sum([ mot[0] for mot in scorepos]),'neg':np.sum([mot[1] for mot in scorepos]),
 'flag':np.sum([mot[2] for mot in scorepos])}
   # return scorepos

On calcule ensuite le nouveau vecteur de score 

In [514]:
taggedTweetscore=taggedTweet.apply(lambda x: scoretweetmodif(x))



On récupère les termes négatifs contenus dans des tweets positifs dans une liste

In [ ]:

listneg=[ x for x in taggedTweetscore if (x['flag'] > 0 and x['pos'] > x ['neg']) ]

On calcule enfin le score et la matrice de confusion 

In [515]:
taggedTweetscore=taggedTweet.apply(lambda x: scoretweetmodif(x))
taggedTweetClass=taggedTweetscore.apply( lambda x: classscore(x))
predctedscore=taggedTweetClass.apply(lambda x:x[2])
df_confusion = pd.crosstab(df.score, predctedscore, rownames=['Actual'], colnames=['Predicted'], margins=True)


In [516]:
df_confusion

Predicted,0,2,4,All
Actual,,,,
0,80,29,68,177
2,19,58,62,139
4,26,29,127,182
All,125,116,257,498


Le nombre de tweets negatifs ,neutres et positifs correctement
détectés avec cette version de l’algorithme sont respectivement **80** , **58** et **127**

In [519]:
score=(df_confusion[0:1][0].values[0] +df_confusion[1:2].values[0][1] +df_confusion[2:3].values[0][2])/df_confusion[3:4].values[0][3]
print("Le score de cette version est %.2f" %score)

Le score de cette version est 0.53


On est arrivé à améliorer le score de 4% , ce qui n'est pas mal . On constate , via la matrice de confusion, que le traitement des négations et des modifieurs a permis de réduire l'erreur qu'on commet sur la classe 4 : L'algorithme a tendance au début de classifier les tweets en positifs , mais après ce traitement , il arrive à distinguer mieux le neutre et le négatif du positif . Essayons maintenant de creuser d'autres pistes 

In [523]:
listneg=[ x for x in taggedTweetscore if (x['flag'] > 0 and x['pos'] > x ['neg']) ]
print( "Le nombre de termes négatifs contenus dans les tweets positifs est %s" %(len(listneg)))

Le nombre de termes négatifs contenus dans les tweets positifs est 4


### 5.  Algorithme de détection v3 : gestion des emoticons


Maintenant , on va essayer de gérer les emoticons dans les tweets en utilisant le dictionnaire d’émoticons `EmoticonLookupTable.txt`

On va garder le vecteur de score obtenu avec la partie précédente et on contruisera également un autre score uniquement basé sur les emoticons , sur la base initiale de tweets , mais en supprimant les URL car elles contiennent `:/` .
- Les émoticons positifs rencontrés augmentent de 1 la valeur du score positif du tweet
- Les émoticons négatifs augmentent de 1 la valeur du score négatif du tweet.


On commence par lire le fichier des emoticons et le transformer en dataframe

In [117]:
dictfile="EmoticonLookupTable.txt"
with open(dictfile) as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)
    


In [118]:
emoticon=pd.DataFrame(d)
emoticon.columns=["symbol","score"]

In [119]:
emoticon.head()

,symbol,score
0,%-(,-1
1,%-),1
2,(-:,1
3,(:,1
4,(^ ^),1


On transforme la colonne score en numérique

In [521]:
emoticon.score = pd.to_numeric(emoticon.score, errors='coerce')


On reparte de la base df_temp qui contient les tweets initiaux après avoir supprimer les urls

In [522]:
df_temp.head(3)

,score,id,date,req,user,content
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."


On développe ensuite une fonction qui renvoie le score de chaque tweet en se basant uniquement sur les emoticons

In [122]:
def getsmileyscore ( tweet) :
    scorep,scoren=0,0
    for i,emo in enumerate(emoticon.symbol.tolist()) :
        if emo in tweet :
            if int(emoticon.score[i]) > 0 :
                scorep +=1
            else : 
                scoren +=1 
    return {'emop':scorep ,'emon':scoren}
                

Ci-desous un exemple d'output de la fonction

In [524]:
tweet="Reading my kindle2... Love it... Lee childs i... :/"
getsmileyscore(tweet)

{'emop': 0, 'emon': 1}

On calcule ensuite le emoticon_score de tous les tweets :

In [525]:
smiley_score=df_temp.content.apply(lambda x: getsmileyscore(x))


On combine à la fin  le score de v2 et le score d'emoticon pour calcule rle score final :

In [528]:
taggedTweetscoresmiley=taggedTweetscore.copy()
for  i,el in enumerate(taggedTweetscore) :
    c=taggedTweetscore[i].copy()
    taggedTweetscoresmiley[i]['pos']=taggedTweetscoresmiley[i]['pos']+smiley_score[i]['emop']
    taggedTweetscoresmiley[i]['neg']=taggedTweetscoresmiley[i]['neg']+smiley_score[i]['emon']
   # if c['pos']> c['neg'] and  (taggedTweetscoresmiley[i]['pos'] < taggedTweetscoresmiley[i]['neg'] or taggedTweetscoresmiley[i]['pos']== taggedTweetscoresmiley[i]['neg']):
    #    print([i])

Ci-dessous un extraint du vecteur de score obtenu : 

In [529]:
taggedTweetscoresmiley[0:5]

0     {'pos': 1.125, 'neg': 1.25, 'flag': 0}
1    {'pos': 1.625, 'neg': 0.125, 'flag': 0}
2        {'pos': 0.0, 'neg': 0.0, 'flag': 0}
3        {'pos': 4.0, 'neg': 2.0, 'flag': 1}
4        {'pos': 2.0, 'neg': 0.0, 'flag': 0}
Name: content, dtype: object

Calculons maintenant la nouvelle matrice de confusion et le score de l'algorithme

In [531]:

taggedTweetClassSmiley=taggedTweetscoresmiley.apply( lambda x: classscore(x))
predctedscoreSmiley=taggedTweetClassSmiley.apply(lambda x:x[2])
df_confusion_smiley = pd.crosstab(df.score, predctedscoreSmiley, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion_smiley

Predicted,0,2,4,All
Actual,,,,
0,88,27,62,177
2,19,58,62,139
4,26,24,132,182
All,133,109,256,498


Le nombre de tweets negatifs ,neutres  et positifs correctement
détectés avec cette version de l’algorithme sont respectivement **88** , **58** et **132**

In [533]:
score=(df_confusion_smiley[0:1][0].values[0] +df_confusion_smiley[1:2].values[0][1] +df_confusion_smiley[2:3].values[0][2])/df_confusion_smiley[3:4].values[0][3]
score
print("Le score de version V3 est %.2f" %score)

Le score de version V3 est 0.56


In [540]:
nbre_emoticon=np.sum([smiley['emop'] + smiley['emon'] for smiley in smiley_score])
print("Le nombre d'emoticons  est %s" %nbre_emoticon)

Le nombre d'emoticons  est 57


Le fait de tenir compte des emoticons nous a permis d'améliorer le score de 3 % . Etant donnée le nombre faible d'emoticons dans le corpus , on ne peut pas esperer une amélioration significative .
On voit également qu'on commet une erreur importante dans la distinction entre la classe neutre et la classe positive

### 6 . Algorithme de détection v4

Pour améliorer notre score , on suggère d'envisager une approche sur deux étapes  : 
- La première  consiste à enrichir les dictionnaires de detection de sentiments et le traitmeent de ponctuation
- La deuxième consiste à utiliser un corpus existant de tweet labélisé 'positf' et 'negatif' disponible sur intrernet , entrainer un classifieur la dessus , ensuite prédire le score de chaque tweet de notre dataset ( proba d'etre positif ) et enfin ajouter la probabilité ( si elle est très elevé ou très faible )  au score obtenu de l'algorithme v3 

#### Première étape  : enrichissement et ajout des dicitionnaires 

En premier lieu , on a commencé par enrichir les dictionnaires de négations et de modifieurs utilisés précedemment vu qu'ils comportent un nombre faible d'élements . Pour ce faire : 
- on a mis à jour le dictionnaire des termes négatifs
- on a ajouté un autre dictionnaire d'adverbes scrappé du https://en.wiktionary.org/wiki/Category:English_degree_adverbs
- on a ajouté aussi les termes de ponctuations accentués comme '!!!' ou '???' qui pourrait avoir le meme effet que les adverbes 

In [541]:
negatives = ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
     "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
     "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
     "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
     "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
     "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
     "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
     "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite","aren't","arent","can't","cannot","cant","don't","dont","isn't","isnt","never","not","won't",
           "wont","wouldn't","wouldnt"]

In [542]:
BOOSTER_DICT = ["absolutely", "amazingly","awfully", "completely", "considerably",
     "decidedly", "deeply", "effing", "enormously",
     "entirely", "especially", "exceptionally", "extremely",
     "fabulously", "flipping", "flippin",
     "fricking", "frickin", "frigging", "friggin", "fully", "fucking",
     "greatly", "hella", "highly" , "hugely", "incredibly",
     "intensely", "majorly", "more", "most", "particularly",
     "purely", "quite", "really", "remarkably",
     "so", "substantially",
     "thoroughly", "totally", "tremendously",
     "uber", "unbelievably", "unusually", "utterly",
     "very","too","quite",
     "almost", "barely", "hardly" , "just enough",
     "kind of", "kinda", "kindof", "kind-of",
     "less", "little", "marginally", "occasionally", "partly",
     "scarcely", "slightly", "somewhat",
     "sort of", "sorta", "sortof", "sort-of"]
PUNC_LIST = ["!!", "!!!", "??", "???", "?!?", "!?!", "?!?!", "!?!?"]

On a adapté ensuite nos fonctions de scoring pour qu'ils tiennent compte des nouveaux dictionnaires

In [543]:
def extractscoremotmodif (mot,prec):
    tt=wn.synsets(mot)
    coef=1
    if len(wn.synsets(mot)) > 0 :
        if prec in (modifierslist+BOOSTER_DICT+PUNC_LIST): 
            coef=2
        tt=tt[0].name() # à exploiter les autre options
        s=swn.senti_synset(tt)
        if prec in negatives :
            return coef*s.neg_score(),coef*s.pos_score(),1
            
        else :
            return coef*s.pos_score(),coef*s.neg_score(),0
    
    else :
        return 0,0,0

def scoretweetmodif (tweettag):

    scorepos=[]
    listnouns=[ x for (x,y) in tweettag if y in listkeep]

    for ind,mot in enumerate(listnouns) :
        if ind > 0 :
            score = extractscoremotmodif (mot,listnouns[ind-1])
        else :
            score =extractscoremot(mot)
            score=(score[0],score[1],0)
        scorepos.append(score)
    return {'pos':np.sum([ mot[0] for mot in scorepos]),'neg':np.sum([mot[1] for mot in scorepos]),
 'flag':np.sum([mot[2] for mot in scorepos])}
   # return scorepos

In [547]:

####### une fonction qui calcule le score à partir d'une matrice de confusion  ############
def getscore ( dfconfusion) :
    score=(dfconfusion[0:1][0].values[0] +dfconfusion[1:2].values[0][1] +dfconfusion[2:3].values[0][2])/dfconfusion[3:4].values[0][3]
    return score
#taggedTweetscoremodif1=taggedTweet_beforv3.apply(lambda x: scoretweetmodif(x))


#### Deuxième étape  : Modèle sur un corpus labélisé + et - 

Comme on l'a précisé au début de la section , notre idée consiste à utiliser un jeux de données disponible sur internet qui contient des tweets labélisés { positif , négatif } , générer un embedding des tweets  et ensuite entrainer un classifieur permettant d'estimer la probabilité qu'un tweet soit positif. Ensuite on utilise ce classifieur pour prédire les classe de tweet de notre jeux de données ( en probabilité )  : 
- si la probabilité d'etre positif est supérieur à 0.7 , on ajoute un +1 au score positif du tweet obtenu de l'algorithme v3
- si la probabilité d'etre positif est inférieur à 0.3 , on ajoute un +1 au score négatif obtenu de l'algorithme v3

Le dataset 'Sentiment Analysis Dataset.csv' peut etre téléchargé via le lien suivant : http://thinknook.com/wp-content/uploads/2012/09/Sentiment-Analysis-Dataset.zip


On commence par lire les données : 

In [11]:
data = pd.read_csv('Sentiment Analysis Dataset.csv',
                       usecols=['Sentiment', 'SentimentText'], error_bad_lines=False)

    

In [145]:
data.head(25)

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...
5,0,or i just worry too much?
6,1,Juuuuuuuuuuuuuuuuussssst Chillin!!
7,0,Sunny Again Work Tomorrow :-| ...
8,1,handed in my uniform today . i miss you ...
9,1,hmmmm.... i wonder how she my number @-)


Ici on définit quelques fonctions permettant de traiter les tweets de ce dataset 

In [553]:
def html2unicode(s):
    # These are for regularizing HTML entities to Unicode:
    html_entity_digit_re = re.compile(r"&#\d+;")
    html_entity_alpha_re = re.compile(r"&\w+;")

    # First the digits:
    ents = set(html_entity_digit_re.findall(s))
    if len(ents) > 0:
        for ent in ents:
            entnum = ent[2:-1]
            try:
                entnum = int(entnum)
                s = s.replace(ent, unichr(entnum))
            except:
                pass

    # Now the alpha versions:
    ents = set(html_entity_alpha_re.findall(s))
    ents = filter((lambda x : x != "&amp;"), ents)

    for ent in ents:
        entname = ent[1:-1]
        try:
            s = s.replace(ent, unichr(html.entities.name2codepoint[entname]))
        except:
            pass
        s = s.replace("&amp;", " and ")

    return s


In [554]:
# The code below was adapted from Chris Potts' implementation at
# http://sentiment.christopherpotts.net/code-data/happyfuntokenizing.py

import re
#import htmlentitydefs
import html.entities 

def get_regex_strings():
    return (
        # Phone numbers:
        r"""
        (?:
          (?:            # (international)
            \+?[01]
            [\-\s.]*
          )?
          (?:            # (area code)
            [\(]?
            \d{3}
            [\-\s.\)]*
          )?
          \d{3}          # exchange
          [\-\s.]*
          \d{4}          # base
        )"""
        ,
        # Emoticons:
        r"""
        (?:
          [<>]?
          [:;=8]                     # eyes
          [\-o\*\']?                 # optional nose
          [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
          |
          [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
          [\-o\*\']?                 # optional nose
          [:;=8]                     # eyes
          [<>]?
        )"""
        ,
        # HTML tags:
         r"""<[^>]+>"""
        ,
        # Twitter username:
        r"""(?:@[\w_]+)"""
        ,
        # Twitter hashtags:
        r"""(?:\#+[\w_]+[\w\'_\-]*[\w_]+)"""
        ,
        # Remaining word types:
        r"""
        (?:[a-z][a-z'\-_]+[a-z])       # Words with apostrophes or dashes.
        |
        (?:[+\-]?\d+[,/.:-]\d+[+\-]?)  # Numbers, including fractions, decimals.
        |
        (?:[\w_]+)                     # Words without apostrophes or dashes.
        |
        (?:\.(?:\s*\.){1,})            # Ellipsis dots.
        |
        (?:\S)                         # Everything else that isn't whitespace.
        """
        )


def html2unicode(s):
    # These are for regularizing HTML entities to Unicode:
    html_entity_digit_re = re.compile(r"&#\d+;")
    html_entity_alpha_re = re.compile(r"&\w+;")

    # First the digits:
    ents = set(html_entity_digit_re.findall(s))
    if len(ents) > 0:
        for ent in ents:
            entnum = ent[2:-1]
            try:
                entnum = int(entnum)
                s = s.replace(ent, unichr(entnum))
            except:
                pass

    # Now the alpha versions:
    ents = set(html_entity_alpha_re.findall(s))
    ents = filter((lambda x : x != "&amp;"), ents)

    for ent in ents:
        entname = ent[1:-1]
        try:
            s = s.replace(ent, unichr(html.entities.name2codepoint[entname]))
        except:
            pass
        s = s.replace("&amp;", " and ")

    return s


def tokenize(s):
    # This is the core tokenizing regex:
    word_re = re.compile(r"""(%s)""" % "|".join(get_regex_strings()), re.VERBOSE | re.UNICODE)

    # The emoticon string gets its own regex so that we can preserve case for them as needed:
    emoticon_re = re.compile(get_regex_strings()[1], re.VERBOSE | re.I | re.UNICODE)

    # Try to ensure unicode:


    # Fix HTML character entitites:
    s = html2unicode(s)

    # Tokenize:
    words = word_re.findall(s)

    # Possible alter the case, but avoid changing emoticons like :D into :d:
    words = map((lambda x : x if emoticon_re.search(x) else x.lower()), words)

    return words


On génère ensuite un embedding  tf-idf des tweet prétraités  via la commande `TfidfVectorizer`

In [555]:
print('Pre-processing tweet text...')
corpus = data['SentimentText']
vectorizer = TfidfVectorizer(decode_error='replace', strip_accents='unicode',
                                 stop_words='english', tokenizer=tokenize)
X = vectorizer.fit_transform(corpus.values)
y = data['Sentiment'].values

Pre-processing tweet text...


On entraine ensuite un modèle **NaiveBayes** sur l'embedding des tweets  ( c'est le modèle basique , on pourra également essayer des approches plus sophistiquées comme les CNN et les RNN )

In [556]:
print('Training sentiment classification model...')
classifier = MultinomialNB()
classifier.fit(X, y)

Training sentiment classification model...


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Le score du classifieur sur le train est 83% , on pourra certainement l'améliorer mais , par faute de temps , on ne va pas creuser ce point .

In [557]:
classifier.score(X,y)

0.8324175510922873

Maintenant , on recupère les données de notre dataset et on applique notre vectorizer pour générer l'embedding correspondant : 

In [559]:
X_predict = vectorizer.transform(df_temp.content)


On prédit ensuite la probabilité d'avoir un sentiment positif de chaque tweet 

In [560]:
proba_model=classifier.predict_proba(X_predict)

In [561]:
score_model_pos=[ i[1] for i in proba_model]
score_model_pos

[0.5951628792743565,
 0.6165334225115324,
 0.6304053832021198,
 0.6200921456994368,
 0.6633596009306225,
 0.6843895971415599,
 0.11603542840026963,
 0.7185028645747751,
 0.765744695307406,
 0.6402629234268521,
 0.685392924249912,
 0.4365668207043725,
 0.6640085296999537,
 0.47029888887596905,
 0.2004342063906284,
 0.6183404144764137,
 0.20346297238884145,
 0.47563211725327753,
 0.41368438115792266,
 0.5285190717810705,
 0.6826535724485436,
 0.6518773687300844,
 0.5368696261540217,
 0.45841869516094863,
 0.3187564600924987,
 0.888466570555584,
 0.5043652116971936,
 0.7927869444936917,
 0.7621391251215481,
 0.7256761312331165,
 0.5598433590332862,
 0.7313032057001897,
 0.6155361288957714,
 0.30317204305859086,
 0.43398149971390554,
 0.21932919592317157,
 0.04353630944826192,
 0.2327365586202071,
 0.6244935318782446,
 0.5402898620329991,
 0.6347944149261476,
 0.49715395595128664,
 0.06262348325185872,
 0.17921457385602513,
 0.25737920418891813,
 0.42556571916782704,
 0.5000500885211578,
 

On recupère après les scores des tweets de notre dernier algorithme du TP 

In [156]:
taggedTweetscoresmileymodel=taggedTweetscoresmiley.copy()

On met à jour les scores en tenant compte de la probabilité issu du modèle NaiveBayes : 
- si la probabilité d'etre positif est supérieur à 0.7 , on ajoute un +1 au score positif du tweet obtenu de l'algorithme v3
- si la probabilité d'etre positif est inférieur à 0.3 , on ajoute un +1 au score négatif obtenu de l'algorithme v3



In [562]:
#taggedTweetClassSmiley=taggedTweetscoresmiley.apply( lambda x: classscore(x))
for i,s in enumerate(taggedTweetscoresmiley) :
    if score_model_pos[i] > 0.7 :
        taggedTweetscoresmileymodel[i]['pos']=taggedTweetscoresmiley[i]['pos']+1
    
    if score_model_pos[i] < 0.3 :
        taggedTweetscoresmileymodel[i]['neg']=taggedTweetscoresmiley[i]['neg']+1



On calcule enfin la matrice de confusion  et le score 

In [563]:
taggedTweetClassSmileyModel=taggedTweetscoresmileymodel.apply( lambda x: classscore(x))
predctedscoreSmileymodel=taggedTweetClassSmileyModel.apply(lambda x:x[2])
df_confusion_smileymodel = pd.crosstab(df.score, predctedscoreSmileymodel, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion_smileymodel

Predicted,0,2,4,All
Actual,,,,
0,132,10,35,177
2,20,52,67,139
4,17,13,152,182
All,169,75,254,498


In [565]:
score=(df_confusion_smileymodel[0:1][0].values[0] +df_confusion_smileymodel[1:2].values[0][1] +df_confusion_smileymodel[2:3].values[0][2])/df_confusion_smileymodel[3:4].values[0][3]
score
print( "Le score du dernier algo est %.2f" %score)

Le score du dernier algo est 0.67


On arrive à améliorer le score initial de presque 10% , ce qui n'est pas mal de tout par rapport à l'effort fourni sur la partie modélisation .On arrive surtout à distinguer le neutre du positif via cette approche ( dans v3 , il y avait 62 neutres qui sont classés positifs , et maintenant ça devient 32 ) .
Pour continuer à améliorer le score , on pourra enrichir le dictionnaires des emoticons , utiliser des dictionnaires existant des mots +/- , des traducteurs de phrases ( parfois on peut observer des mots en langues étrangères ) , des termes anglais courants ...